In [1]:
import psycopg2
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib

latex = False

if latex:
    matplotlib.use("pgf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })

In [2]:
MIN_LOS_ICU = 24

# Load Data

### From SQL

In [3]:
# Connect to db
conn = psycopg2.connect(host='localhost', port=5432, dbname='mimic', user='zainab', password='password')
cur = conn.cursor() 

# Read vital signs
vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vital_resampled_min{MIN_LOS_ICU:d}h;', conn)

# Read in labs values
labs = pd.read_sql_query(f'SELECT * FROM mimiciii.lab_resampled_min{MIN_LOS_ICU:d}h;', conn)

# Read demographics
demographics = pd.read_sql_query(f'SELECT * FROM mimiciii.demographics_min{MIN_LOS_ICU:d}h;', conn)

# Close the cursor and connection to so the server can allocate bandwidth to other requests
cur.close()
conn.close()

/tmp/ipykernel_85013/4266879397.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vital_resampled_min{MIN_LOS_ICU:d}h;', conn)
/tmp/ipykernel_85013/4266879397.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  labs = pd.read_sql_query(f'SELECT * FROM mimiciii.lab_resampled_min{MIN_LOS_ICU:d}h;', conn)
/tmp/ipykernel_85013/4266879397.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  demographics = pd.read_sql_query(f'SELECT * FROM mimiciii.demographics_min{MIN_LOS_ICU

### From File

In [4]:
data_path = f'../data/min{MIN_LOS_ICU:d}h/'

demographics = pd.read_pickle(data_path + f'demographics_min{MIN_LOS_ICU:d}h.pickle')
vitals = pd.read_pickle(data_path + f'vitals_min{MIN_LOS_ICU:d}h.pickle')
labs = pd.read_pickle(data_path + f'labs_min{MIN_LOS_ICU:d}h.pickle')

# Preprocess data:

In [5]:
demographics['los_icu_hours'] =     demographics['los_icu'] * 24

demographics['los_icu_bin'] =       pd.cut(demographics['los_icu_hours'],
                                        bins=[8, 15, 23, 31, 39, 47, 55, 63, 71, max(demographics['los_icu_hours'])],
                                        labels=['8-15', '16-23', '24-31', '32-39', '40-47', '48-55', '56-63', '64-71', '>72']
                                    ).astype(str)

demographics['admission_age_bin'] = pd.cut(demographics['admission_age'],
                                        bins=[0, 29, 59, 89, max(demographics['admission_age'])],
                                        labels=['0-29', '30-59', '60-89', '>90']
                                    ).astype(str)

ValueError: bins must increase monotonically.

# Analysis

## Basic data description and overview

Describe vitals

In [6]:
vitals.head(5)

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,label_death_icu
0,201204,2121-12-07 20:00:00,78.0,160.0,20.0,59.0,18.5,34.777778,97.5,0
1,201204,2121-12-07 21:00:00,76.5,154.0,52.0,69.0,18.5,34.777778,97.5,0
2,201204,2121-12-07 22:00:00,72.0,131.0,50.0,68.0,21.0,34.777778,100.0,0
3,201204,2121-12-07 23:00:00,71.5,171.0,55.0,79.0,18.5,34.777778,99.0,0
4,201204,2121-12-08 00:00:00,71.0,141.0,67.0,83.0,27.0,35.555556,96.0,0


In [7]:
vitals.describe()

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,label_death_icu
count,2116.000000,2116,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000
mean,247489.568998,2155-07-26 18:45:13.610586112,85.269849,123.761342,59.267604,77.858578,19.699315,36.616879,96.091682,0.146503
min,201204.000000,2102-08-31 17:00:00,31.500000,50.000000,17.000000,23.500000,3.000000,33.999998,46.000000,0.000000
25%,219013.000000,2130-02-05 03:45:00,74.000000,107.000000,50.000000,68.333298,15.000000,36.111111,95.000000,0.000000
50%,243238.000000,2151-09-14 12:30:00,84.000000,122.000000,59.000000,77.166649,19.000000,36.555557,97.000000,0.000000
75%,277021.000000,2185-03-25 08:15:00,95.000000,140.000000,69.000000,87.000000,23.000000,37.111113,99.000000,0.000000
max,296804.000000,2198-11-01 07:00:00,149.000000,201.000000,137.000000,146.000000,63.000000,40.000000,100.000000,1.000000
std,28821.435954,NaN,15.156675,23.797088,14.897067,14.794135,5.982945,0.713099,5.043227,0.353693


Describe labs

In [8]:
labs.head(5)

,icustay_id,charttime,albumin,bun,bilirubin,lactate,bicarbonate,bands,chloride,creatinine,glucose,hemoglobin,hematocrit,platelet,potassium,ptt,sodium,wbc,label_death_icu
0,201204,2121-12-07 20:00:00,-1.0,97.0,-1.0,1.4,23.0,-1.0,106.0,3.2,193.0,8.9,26.3,286.0,4.7,28.7,141.0,8.8,0
1,201204,2121-12-08 04:00:00,-1.0,91.0,-1.0,1.4,24.0,-1.0,108.0,2.6,97.0,9.7,27.6,261.0,4.5,25.9,143.0,9.8,0
2,201204,2121-12-08 12:00:00,-1.0,82.0,-1.0,1.4,28.0,-1.0,109.0,2.3,112.0,9.7,24.8,261.0,4.0,24.9,145.0,9.8,0
3,201204,2121-12-08 20:00:00,-1.0,80.0,-1.0,1.4,28.0,-1.0,110.0,2.0,126.0,8.6,24.5,274.0,3.8,24.0,146.0,8.9,0
4,201204,2121-12-09 04:00:00,-1.0,80.0,-1.0,1.4,28.0,-1.0,110.0,2.0,126.0,8.6,24.5,274.0,3.8,24.0,146.0,8.9,0


In [9]:
labs.describe()

,icustay_id,charttime,albumin,bun,bilirubin,lactate,bicarbonate,bands,chloride,creatinine,glucose,hemoglobin,hematocrit,platelet,potassium,ptt,sodium,wbc,label_death_icu
count,284.000000,284,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000
mean,247372.334507,2155-05-28 04:01:03.380282368,0.715493,28.542254,0.048944,0.313028,24.755282,-0.116197,104.697183,1.325704,143.508803,10.069366,30.588028,203.380282,4.061268,37.415845,138.920775,11.575704,0.144366
min,201204.000000,2102-08-31 17:00:00,-1.000000,2.000000,-1.000000,-1.000000,12.000000,-1.000000,83.000000,0.300000,43.000000,-1.000000,21.800000,6.000000,2.500000,-1.000000,119.000000,0.700000,0.000000
25%,219013.000000,2130-02-05 10:00:00,-1.000000,13.000000,-1.000000,-1.000000,22.000000,-1.000000,100.000000,0.800000,99.000000,9.337500,27.800000,138.000000,3.700000,27.000000,135.000000,7.500000,0.000000
50%,243238.000000,2151-09-14 08:00:00,-1.000000,21.500000,-1.000000,-1.000000,25.000000,-1.000000,104.000000,1.100000,121.000000,10.100000,30.400000,202.000000,4.000000,33.900000,139.000000,10.300000,0.000000
75%,277021.000000,2185-03-24 20:00:00,2.800000,33.250000,0.500000,1.300000,28.000000,-1.000000,109.000000,1.400000,155.000000,11.150000,33.000000,259.000000,4.362500,43.725000,142.000000,14.200000,0.000000
max,296804.000000,2198-11-01 00:00:00,5.200000,126.000000,14.000000,18.850000,37.000000,13.000000,140.000000,7.200000,773.000000,13.600000,40.600000,448.000000,7.000000,150.000000,165.000000,65.300000,1.000000
std,28714.164080,NaN,2.044463,23.535440,2.006729,2.330726,4.193947,2.614965,7.414878,0.990337,83.834015,1.904410,4.035214,95.795321,0.602495,22.994667,5.886280,8.183653,0.352081


Describe demographics

In [10]:
demographics[['icustay_id','intime','outtime','label_death_icu']].head(5)

,icustay_id,intime,outtime,label_death_icu
0,201204,2121-12-07 20:30:00,2121-12-09 18:43:58,0
1,204881,2149-05-29 18:00:00,2149-05-31 22:19:17,0
2,206504,2164-10-23 22:40:00,2164-10-25 12:21:07,0
3,210164,2127-07-29 08:00:00,2127-07-30 14:11:20,0
4,210474,2142-11-26 22:29:00,2142-11-29 17:36:20,0


In [11]:
demographics.describe()

,icustay_id,subject_id,hadm_id,dod,admittime,dischtime,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,intime,outtime,los_icu,icustay_seq,deathtime_icu,label_death_icu,intime_old,los_icu_hours
count,45.000000,45.000000,45.000000,45,45,45,45.000000,45.000000,45.000000,45.0,45,45,45.000000,45.0,7,45.000000,45,45.000000
mean,248127.022222,26394.222222,149631.577778,2154-10-10 18:08:00,2153-07-23 23:58:33.333332992,2153-07-30 07:44:57.333334016,6.323889,93.301516,0.266667,1.0,2153-07-25 00:02:17.333334016,2153-07-26 20:44:14.044444672,1.862462,1.0,2155-01-03 05:10:00.000001024,0.155556,2153-07-24 23:42:24.599999488,44.699086
min,201204.000000,10006.000000,100375.000000,2105-02-16 00:00:00,2102-08-29 07:15:00,2102-09-06 16:20:00,0.636806,27.873580,0.000000,1.0,2102-08-31 17:09:00,2102-09-01 20:19:42,1.032720,1.0,2105-06-11 02:20:00,0.000000,2102-08-31 17:04:12,24.785278
25%,220671.000000,10046.000000,125449.000000,2130-11-03 00:00:00,2129-05-02 00:12:00,2129-05-06 13:40:00,3.520139,69.591952,0.000000,1.0,2129-05-01 23:00:00,2129-05-03 01:23:24.999999488,1.366019,1.0,2129-03-08 20:44:00,0.000000,2129-05-02 00:12:39.000000512,32.784444
50%,243238.000000,40124.000000,156668.000000,2154-01-22 00:00:00,2151-09-12 17:04:00,2151-09-15 00:45:00,5.401389,78.453534,0.000000,1.0,2151-09-12 20:00:00,2151-09-14 16:53:31.000000512,1.870498,1.0,2163-05-15 12:00:00,0.000000,2151-09-12 18:51:52,44.891944
75%,277021.000000,42135.000000,174997.000000,2182-08-03 00:00:00,2180-07-19 06:55:00,2180-07-20 13:00:00,8.378472,85.736885,1.000000,1.0,2180-07-19 08:27:00,2180-07-20 14:48:44.999999488,2.309074,1.0,2185-09-03 16:41:00,0.000000,2180-07-19 06:56:38,55.417778
max,296804.000000,44222.000000,199207.000000,2202-12-05 00:00:00,2198-10-29 06:54:00,2198-11-05 15:20:00,19.902083,300.003247,1.000000,1.0,2198-10-29 08:18:00,2198-10-31 20:50:58,2.858333,1.0,2186-07-07 19:00:00,1.000000,2198-10-29 06:55:20,68.600000
std,29387.895224,16193.209448,30622.331066,NaN,NaN,NaN,4.135386,66.657475,0.447214,0.0,NaN,NaN,0.540485,0.0,NaN,0.366529,NaN,12.971631


Check if there is empty data

In [12]:
demographics.isnull().sum()

icustay_id                      0
subject_id                      0
hadm_id                         0
gender                          0
dod                             0
admittime                       0
dischtime                       0
los_hospital                    0
admission_age                   0
ethnicity                       0
ethnicity_grouped               0
hospital_expire_flag            0
hospstay_seq                    0
first_hosp_stay                 0
intime                          0
outtime                         0
los_icu                         0
icustay_seq                     0
first_icu_stay_current_hosp     0
first_icu_stay_patient          0
first_careunit                  0
deathtime_icu                  38
label_death_icu                 0
intime_old                      0
los_icu_hours                   0
dtype: int64

In [13]:
labs.isnull().sum().sum()

0

In [14]:
vitals.isnull().sum().sum()

0

## Check timings of measurements:

In [15]:
timings = demographics[['icustay_id', 'intime', 'outtime']].copy()
timings = timings.sort_values('icustay_id').set_index('icustay_id')

Check labs

In [16]:
labs_grouped = labs[['icustay_id', 'charttime']].copy().groupby('icustay_id')
labs_grouped.sort = True

# Calculate offset between first lab and intime:
timings['offset_first_lab'] = labs_grouped.charttime.min()
timings['offset_first_lab'] = (timings['offset_first_lab'] - timings['intime']) / timedelta(hours=1)

# Calculate offset between last lab and outtime:
timings['offset_last_lab'] = labs_grouped.charttime.max()
timings['offset_last_lab'] = (timings['offset_last_lab'] - timings['outtime']) / timedelta(hours=1)

# Calculate number of labs preceding intime:
timings['n_early_labs'] = [(labs_grouped.get_group(i).charttime.to_numpy() < timings.loc[[i]].intime.to_numpy()).sum() for i in timings.index]

# Calculate number of labs after outtime:
timings['n_late_labs'] = [(labs_grouped.get_group(i).charttime.to_numpy() > timings.loc[[i]].outtime.to_numpy()).sum() for i in timings.index]

# Calculate number of labs in between intime and outtime:
timings['n_labs']  = [len(labs_grouped.get_group(i)) for i in timings.index]
timings['n_labs'] -= timings['n_early_labs']
timings['n_labs'] -= timings['n_late_labs']

del labs_grouped

Check vitals

In [17]:
vitals_grouped = vitals[['icustay_id', 'charttime']].copy().groupby('icustay_id')
vitals_grouped.sort = True

# Calculate offset between first vital and intime:
timings['offset_first_vital'] = vitals_grouped.charttime.min()
timings['offset_first_vital'] = (timings['offset_first_vital'] - timings['intime']) / timedelta(hours=1)

# Calculate offset between last vital and outtime:
timings['offset_last_vital'] = vitals_grouped.charttime.max()
timings['offset_last_vital'] = (timings['offset_last_vital'] - timings['outtime']) / timedelta(hours=1)

# Calculate number of vitals preceding intime:
timings['n_early_vitals'] = [(vitals_grouped.get_group(i).charttime.to_numpy() < timings.loc[[i]].intime.to_numpy()).sum() for i in timings.index]

# Calculate number of vitals after outtime:
timings['n_late_vitals'] = [(vitals_grouped.get_group(i).charttime.to_numpy() > timings.loc[[i]].outtime.to_numpy()).sum() for i in timings.index]

# Calculate number of vitals in between intime and outtime:
timings['n_vitals']  = [len(vitals_grouped.get_group(i)) for i in timings.index]
timings['n_vitals'] -= timings['n_early_vitals']
timings['n_vitals'] -= timings['n_late_vitals']

del vitals_grouped

Describe timings

In [18]:
print(
    'All labs and vitals start at the same time: ',
    (timings['offset_first_vital'] == timings['offset_first_lab']).to_numpy().all()
)
timings.describe()

All labs and vitals start at the same time:  True


,intime,outtime,offset_first_lab,offset_last_lab,n_early_labs,n_late_labs,n_labs,offset_first_vital,offset_last_vital,n_early_vitals,n_late_vitals,n_vitals
count,45,45,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
mean,2153-07-25 00:02:17.333334016,2153-07-26 20:44:14.044444672,0.006296,-2.203901,0.311111,0.377778,5.622222,0.006296,1.329432,0.311111,2.911111,43.800000
min,2102-08-31 17:09:00,2102-09-01 20:19:42,-0.500000,-10.950833,0.000000,0.000000,2.000000,-0.500000,-6.784444,0.000000,0.000000,22.000000
25%,2129-05-01 23:00:00,2129-05-03 01:23:24.999999488,-0.150000,-6.732778,0.000000,0.000000,4.000000,-0.150000,-2.151389,0.000000,0.000000,30.000000
50%,2151-09-12 20:00:00,2151-09-14 16:53:31.000000512,0.000000,-2.966944,0.000000,0.000000,6.000000,0.000000,-0.891944,0.000000,0.000000,45.000000
75%,2180-07-19 08:27:00,2180-07-20 14:48:44.999999488,0.150000,1.557778,1.000000,1.000000,7.000000,0.150000,4.575556,1.000000,5.000000,54.000000
max,2198-10-29 08:18:00,2198-10-31 20:50:58,0.500000,11.400000,1.000000,2.000000,9.000000,0.500000,12.163889,1.000000,13.000000,69.000000
std,NaN,NaN,0.268396,5.827671,0.468179,0.613814,1.774682,0.268396,5.056241,0.468179,4.083349,13.062089


In [19]:
timings.head(5)

,intime,outtime,offset_first_lab,offset_last_lab,n_early_labs,n_late_labs,n_labs,offset_first_vital,offset_last_vital,n_early_vitals,n_late_vitals,n_vitals
icustay_id,,,,,,,,,,,,
201204,2121-12-07 20:30:00,2121-12-09 18:43:58,-0.500000,-6.732778,1,0,5,-0.500000,-0.732778,1,0,46
204881,2149-05-29 18:00:00,2149-05-31 22:19:17,0.000000,3.678611,0,1,7,0.000000,6.678611,0,7,53
206504,2164-10-23 22:40:00,2164-10-25 12:21:07,0.333333,-5.351944,0,0,5,0.333333,-2.351944,0,0,36
210164,2127-07-29 08:00:00,2127-07-30 14:11:20,0.000000,-6.188889,0,0,4,0.000000,-1.188889,0,0,30
210474,2142-11-26 22:29:00,2142-11-29 17:36:20,-0.483333,-3.605556,1,0,8,-0.483333,-1.605556,1,0,66


### Overview of the patient's length of stay per bins

In [20]:
icu_length_by_class = demographics.groupby("los_icu_bin")['label_death_icu'].value_counts() 
icu_length_by_class.unstack().plot(kind='bar', stacked= True) 
data_imputed = demographics.drop(columns=['los_icu_bin'])
plt.legend(labels=["discharge","death"])
plt.show()

KeyError: 'los_icu_bin'

## Length of stay in ICU
Focusing on the demographics "los_icu" we see that there is a big difference between the last quantile (90%) and the maximum value. 

In [ ]:
demo_q = demographics['los_icu'].quantile(.9)
print(f"90% Quantile length of stay: {demo_q*24.:.2f}h")

In [ ]:
print("Patients above 90% quantile: \n", demographics[demographics.los_icu > demo_q].label_death_icu.value_counts())
print("Total number of patients: \n", demographics.label_death_icu.value_counts())

## Data distribution for ICU length of stay

Patient's ICU length of stay per hour

In [ ]:
demographics.hist(column='los_icu_hours', bins=np.arange(demographics.los_icu_hours.max()))
plt.ylabel('Nº of patients / hour')
plt.xlabel('Length of ICU stay (hours)')
plt.title("(a) Patient’s ICU length of stay per hour")
#plt.savefig("Exploratory analysis_a.pdf", format="pdf", bbox_inches="tight")
plt.show()

Patient's ICU length of stay for 72h in groups of 8 h

In [ ]:
demographics.hist(column='los_icu_hours', bins=range(8, MIN_LOS_ICU + 49, 8))
plt.ylabel('Nº of patients / 8h')
plt.xlabel('Length of ICU stay (hours)')
plt.title("(b) Patient’s ICU length of stay for 72h in groups of 8h")
#plt.savefig("Exploratory analysis_b.pdf", format="pdf", bbox_inches="tight")
plt.show()

Death patient's ICU length of stay per 1 h

In [ ]:
demographics_death = demographics[demographics.label_death_icu==1]
demographics_death.hist(column='los_icu_hours', bins=np.arange(72), color='red')
plt.ylabel('Nº death of patients / 1h')
plt.xlabel('Length of ICU stay (hours)')
plt.title("(c) Death patient’s ICU length of stay per 1h")
#plt.savefig("Exploratory analysis_c.pdf", format="pdf", bbox_inches="tight")
plt.show()

Death patient's ICU length of stau for 72 h in groups of 8 h 

In [ ]:
demographics_death.hist(column='los_icu_hours', bins=range(8, MIN_LOS_ICU + 49, 8), color='red')
plt.ylabel('Nº of death patients / 8h')
plt.xlabel('Length of ICU stay (hours)')
plt.title("(d) Death patient’s ICU length of stay for 72h in groups of 8h")
#plt.savefig("Exploratory analysis_d.pdf", format="pdf", bbox_inches="tight")
plt.show()

Patient's count for a minimum length of stay of 8 h, 16 h, 24 h, 32 h, 40 h, 48 h, 72 h

In [ ]:
pat_window = pd.DataFrame()

for t in range(8, MIN_LOS_ICU + 48, 8):
    pat_window[f'>={t:d}h'] =  demographics[demographics.los_icu_hours >= t].label_death_icu.value_counts()

pat_window.loc['Total']= pat_window.sum(numeric_only=True, axis=0)
pat_window


## Print patient distribution:

In [ ]:
f = 100. / len(demographics.icustay_id.unique())

def print_distribution(column=None):
    # Actual printing happens here:
    def __print_internal(data):
        if len(data) > 0:
            label_counts = data['label_death_icu'].value_counts()
            latex = ""

            if 1 in label_counts:
                print(f"  Number of ICU deaths:       {label_counts[1]:6d} ({label_counts[1]/label_counts.sum()*100.:5.1f}%)")
                latex += f"& ${label_counts[1]:6,d}$ & ${label_counts[1]*f:5.1f}\%$ "
            else:
                print(f"  Number of ICU deaths:            0 (  0.0%)")
                latex += f"& $     0$ & $  0.0\%$ "

            
            if 0 in label_counts:
                print(f"  Number of ICU discharges:   {label_counts[0]:6d} ({label_counts[0]/label_counts.sum()*100.:5.1f}%)")
                latex += f"& ${label_counts[0]:6,d}$ & ${label_counts[0]*f:5.1f}\%$ "
            else:
                print(f"  Number of ICU discharges:        0 (  0.0%)")
                latex += f"& $     0$ & $  0.0\%$ "

            latex += f"& ${label_counts.sum():6,d}$ "
            print(f"Latex: '{latex:s}'")

    # Print overall class distribution if column is None:
    if column == None:
        __print_internal(demographics)

    # Print class distribution of column:
    else:
        for key in demographics[column].unique():
            print(f"'{key:s}':")
            __print_internal(demographics[demographics[column]==key])
            print()

In [ ]:
print_distribution()

Print gender distribution:

In [ ]:
print_distribution('gender')

Print ethnicity distribution:

In [ ]:
print_distribution('ethnicity_grouped')

Print age distribution:

In [ ]:
print_distribution('admission_age_bin')